In [116]:
import numpy as np
import pandas as pd

from sklearn.cluster import KMeans
from kneed import KneeLocator
from tqsdk import TqApi,TqAuth
api=TqApi(web_gui=True,auth=TqAuth('18049858285','18049858285wk'))

2024-09-24 14:37:03 -     INFO - 您可以访问 http://127.0.0.1:60864 查看策略绘制出的 K 线图形。


In [117]:
ticks=api.get_tick_serial("SHFE.rb2501")
klines=api.get_kline_serial("SHFE.rb2501",900)

2024-09-24 14:37:11 -     INFO - 通知 : 与 wss://api.shinnytech.com/t/nfmd/front/mobile 的网络连接已建立
2024-09-24 18:11:11 -  WARNING - 通知 : 与 wss://api.shinnytech.com/t/nfmd/front/mobile 的网络连接断开，请检查客户端及网络是否正常
2024-09-24 18:11:11 -  WARNING - 通知 : 开始与 wss://api.shinnytech.com/t/nfmd/front/mobile 的重新建立网络连接


In [120]:
def remove_close_numbers(numbers,threshold):
    result=[numbers[0]]
    for i in range(1,len(numbers)):
        if abs(numbers[i]-result[-1]) > threshold:
            result.append(numbers[i])
    
    return result

def find_closest_numbers(numbers,target,k):
        differences=[abs(number-target) for number in numbers]
        sorted_diff=sorted(zip(numbers,differences),key=lambda x:x[1])
        closet_numbers=[number for number, _ in sorted_diff[:k]]
        return closet_numbers

while True:
    api.wait_update()
    if api.is_changing(klines.iloc[-2],"close"):
        fifmin_close=klines['close'][-61:-1]
        x=np.array(fifmin_close)
        sum_of_sq_instances=[]
        K=range(1,10)
        for k in K:
            km=KMeans(n_clusters=k)
            km=km.fit(x.reshape(-1,1))
            sum_of_sq_instances.append(km.inertia_)
        kmeans=KMeans(n_clusters=3).fit(x.reshape(-1,1))
        c=kmeans.predict(x.reshape(-1,1))
        min_and_max=[]
        for i in range(kn.knee):
            min_and_max.append([-np.inf,np.inf])
        for i in range(len(x)):
            cluster=c[i]
            if x[i] > min_and_max[cluster][0]:
                min_and_max[cluster][0] = x[i]
            if x[i] < min_and_max[cluster][1]:
                min_and_max[cluster][1] = x[i]
        mam_arr=np.sort((np.array(min_and_max)).flatten())
        sup=remove_close_numbers(mam_arr,10)
        bi_sup=find_closest_numbers(numbers=sup,target=3600,k=2)
        km_uplines=bi_sup[0]
        km_dnlines=bi_sup[1]
        print(km_uplines)
        print(km_dnlines)